# Modeling

In [1]:
import gzip
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
from utils.signal_processing import *
%matplotlib inline
plt.style.use('ggplot')

DATA_PATH = '/root/Workspace/DataWarehouse/stMary_RRpo'

In [2]:
with gzip.open(f'{DATA_PATH}/21_230518_resamp_sliced125_filt_patient_stmary.pickle.gzip', 'rb') as f:
    dataset = pickle.load(f)

print(len(dataset), len(dataset[0][0]))

random.seed(42)
random.shuffle(dataset)

pleths = []
resps = []
for ppg, rr in dataset:
    pleths.append(ppg.astype(np.float64))
    resps.append(rr)

pleths = np.asarray(pleths)
resps = np.asarray(resps)
print(pleths.shape, resps.shape)

6508 1800
(6508, 1800) (6508,)


## Normalization

In [3]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

In [4]:
scaler = MinMaxScaler()
scaled_pleths = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in pleths])
print(scaled_pleths.shape, type(scaled_pleths[0][0][0]))

ratio_tr = 0.8
train_x, train_y = scaled_pleths[:int(len(scaled_pleths)*ratio_tr)], resps[:int(len(resps)*ratio_tr)]
val_x, val_y = scaled_pleths[int(len(scaled_pleths)*ratio_tr):], resps[int(len(resps)*ratio_tr):]
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)

(6508, 1800, 1) <class 'numpy.float64'>
(5206, 1800, 1) (5206,)
(1302, 1800, 1) (1302,)


## Architecture: Unet

- Unet 모델 구조를 이용한다.
- Level을 너무 깊게 파지 말고 천천히 만들어보자.

In [5]:
import os
import keras
import tensorflow as tf
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Conv1D, Conv1DTranspose, MaxPooling1D, AveragePooling1D, Dense, BatchNormalization, Activation, Add, Flatten, Dropout, Concatenate
print(f'Is GPU Avaliable: {tf.config.list_physical_devices("GPU")}')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2023-05-25 11:09:03.253546: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 11:09:03.293789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 11:09:03.966739: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
class ContractionBlock(Model):
    def __init__(self, starting_filter=32, dropout=0, *args, **kwargs):
        super(ContractionBlock, self).__init__(*args, **kwargs)
        self.conv1a = Conv1D(filters=starting_filter, kernel_size=3, strides=1, padding='same')
        self.conv1b = Conv1D(filters=starting_filter*2, kernel_size=3, strides=1, padding='same')
        self.maxpool = MaxPooling1D(pool_size=2, strides=2, padding='same')
        
        if dropout!=0:
            self.dropout = Dropout(rate=dropout)
        else:
            self.dropout = None


    def call(self, inputs, training=None, mask=None):
        feature_map = self.conv1a(inputs)
        feature_map = Activation('relu')(feature_map)
        feature_map = self.conv1b(feature_map)
        feature_map = Activation('relu')(feature_map)
        
        if self.dropout is not None:
            feature_map = self.dropout(feature_map)

        outputs = self.maxpool(feature_map)

        return outputs, feature_map

In [7]:
class BottleNeckBlock(Model):
    def __init__(self, starting_filter=512, dropout=0, *args, **kwargs):
        super(BottleNeckBlock, self).__init__(*args, **kwargs)
        self.conv1a = Conv1D(filters=starting_filter, kernel_size=3, strides=1, padding='same')
        self.conv1b = Conv1D(filters=starting_filter*2, kernel_size=3, strides=1, padding='same')
        self.dropout = Dropout(dropout)


    def call(self, inputs, training=None, mask=None):
        outputs = self.conv1a(inputs)
        outputs = Activation('relu')(outputs)
        outputs = self.conv1b(outputs)
        outputs = Activation('relu')(outputs)
        outputs = self.dropout(outputs)
        
        return outputs

In [8]:
class ExpansionBlock(Model):
    def __init__(self, starting_filter=512, *args, **kwargs):
        super(ExpansionBlock, self).__init__(*args, **kwargs)
        self.conv1a = Conv1D(filters=starting_filter*2, kernel_size=3, strides=1, padding='same')
        self.conv1b = Conv1D(filters=starting_filter, kernel_size=3, strides=1, padding='same')
        self.upsampling = Conv1DTranspose(filters=starting_filter, kernel_size=2, strides=2, padding='same')
    

    def call(self, inputs, feature_map, training=None, mask=None):
        x = self.upsampling(inputs)
        x = Concatenate(axis=2)([x, feature_map])
        x = self.conv1a(x)
        x = Activation('relu')(x)
        x = self.conv1b(x)
        x = Activation('relu')(x)

        return x

In [9]:
# output, feature_map = ContractionBlock(starting_filter=32, dropout=0.5)(val_x)
# print(output.shape, feature_map.shape)
# output = BottleNeckBlock(starting_filter=64, dropout=0.5)(output)
# print(output.shape)
# output = ExpansionBlock(starting_filter=64)(output, feature_map)
# print(output.shape)

In [10]:
class Unet(Model):
    def __init__(self, *args, **kwargs):
        super(Unet, self).__init__(*args, **kwargs)
        self.contraction_blocks = [ContractionBlock(starting_filter=f, dropout=d) for f, d in [(32,0), (64,0), (128,0), (256,0.5)]]
        self.bottleneck_block = BottleNeckBlock(starting_filter=512, dropout=0.5)
        self.expansion_blocks = [ExpansionBlock(starting_filter=f) for f in [512, 256, 128, 64]]
        self.avgpool = AveragePooling1D(pool_size=2, strides=2, padding='valid')
        self.maxpool = MaxPooling1D(pool_size=2, strides=2, padding='valid')

        self.d100 = Dense(100, activation='relu')
        self.d50 = Dense(50, activation='relu')
        self.d10 = Dense(10, activation='relu')
        self.d1 = Dense(1)

    @tf.function
    def call(self, inputs, training=None, mask=None):
        x = inputs

        x, feature_map1 = self.contraction_blocks[1](x)
        x, feature_map2 = self.contraction_blocks[2](x)
        x, feature_map3 = self.contraction_blocks[3](x)
        # x, feature_map4 = self.contraction_blocks[3](x)

        x = self.bottleneck_block(x)

        x = self.expansion_blocks[0](x, feature_map3)
        x = self.expansion_blocks[1](x, feature_map2)
        x = self.expansion_blocks[2](x, feature_map1)
        # x = self.expansion_blocks[3](x, feature_map1)


        x = self.avgpool(x)
        x = Flatten()(x)
        x = self.d100(x)
        x = self.d50(x)
        x = self.d10(x)
        
        return self.d1(x)

In [13]:
EPOCHS = 1000
BATCH_SIZE = 256
LR = 0.001
kf = KFold(n_splits=5)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    # ModelCheckpoint('../models/230522-Resnet', monitor='val_loss', save_best_only=True)
]

model = Unet()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    # optimizer=tf.keras.optimizers.SGD(learning_rate=LR, momentum=0.9, weight_decay=0.0001),
    loss=keras.losses.MeanAbsoluteError(),
    metrics=keras.metrics.MeanAbsoluteError()
)

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(BATCH_SIZE)

with tf.device('/GPU:0'):
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        callbacks=callbacks,
        validation_data=val_dataset
    )

Epoch 1/1000


2023-05-25 10:26:54.608499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5206]
	 [[{{node Placeholder/_1}}]]


21/21 [==============================] - ETA: 0s - loss: 16.9733 - mean_absolute_error: 16.9733

2023-05-25 10:27:21.872416: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1302]
	 [[{{node Placeholder/_1}}]]


21/21 [==============================] - 30s 1s/step - loss: 16.9733 - mean_absolute_error: 16.9733 - val_loss: 19.7608 - val_mean_absolute_error: 19.7608 - lr: 0.0100
Epoch 2/1000
21/21 [==============================] - 25s 1s/step - loss: 16.0738 - mean_absolute_error: 16.0738 - val_loss: 14.7659 - val_mean_absolute_error: 14.7659 - lr: 0.0100
Epoch 3/1000
21/21 [==============================] - 25s 1s/step - loss: 13.9460 - mean_absolute_error: 13.9460 - val_loss: 12.6989 - val_mean_absolute_error: 12.6989 - lr: 0.0100
Epoch 4/1000
21/21 [==============================] - 25s 1s/step - loss: 11.8629 - mean_absolute_error: 11.8629 - val_loss: 10.6290 - val_mean_absolute_error: 10.6290 - lr: 0.0100
Epoch 5/1000
21/21 [==============================] - 25s 1s/step - loss: 9.8328 - mean_absolute_error: 9.8328 - val_loss: 8.6948 - val_mean_absolute_error: 8.6948 - lr: 0.0100
Epoch 6/1000
21/21 [==============================] - 25s 1s/step - loss: 8.0073 - mean_absolute_error: 8.0073 -

In [ ]:
min(history.history['val_loss'])

## Load Model

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(128)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(128)

model = keras.models.load_model('../models/230524-Unet/')

In [9]:
pred_y = model.predict(train_dataset)
print(pred_y.shape, train_y.shape)
abs_err = abs(train_y.reshape(-1,1) - pred_y)
print(f'{np.mean(abs_err)} ± {np.std(abs_err)}')

2023-05-25 11:10:58.532212: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5206]
	 [[{{node Placeholder/_1}}]]
2023-05-25 11:10:59.458759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700


41/41 [==============================] - 6s 81ms/step
(5206, 1) (5206,)
0.5693018697474125 ± 0.9266279749787993


In [10]:
pred_y = model.predict(val_dataset)
print(pred_y.shape, val_y.shape)
abs_err = abs(val_y.reshape(-1,1) - pred_y)
print(f'{np.mean(abs_err)} ± {np.std(abs_err)}')

 3/11 [=======>......................] - ETA: 0s

2023-05-25 11:11:16.379412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1302]
	 [[{{node Placeholder/_1}}]]


11/11 [==============================] - 1s 97ms/step
(1302, 1) (1302,)
1.0132102955321562 ± 1.1088444677417446
